In [17]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
:dep num
:dep ndarray
:dep rand
:dep rayon

In [18]:
use num::Complex;
use std::f64::consts::PI;
use ndarray::{Array,Array1,s,azip,Axis};
use plotters::prelude::*;
use rayon::prelude::*;
use rand::Rng;

In [8]:
fn logb(n : usize) -> usize { (n as f32).log(2.0).ceil() as usize }
fn brevidx(i : usize, n : usize) -> usize { if n>0 {i.reverse_bits() >> (64 - n)} else {0} }
fn brevidxs(n : usize) -> Vec<usize> {(0..n).map(|x| brevidx(x,logb(n))).collect::<Vec<_>>()}
fn brev<T: Clone>(a : &[T]) -> Vec<T> { 
   brevidxs(a.len()).iter().map(|i| a[*i].clone()).collect()
}

In [19]:
fn fft(y : &Array1<Complex<f64>>) -> Array1<Complex<f64>> {
    let mut x = y.clone();
    let n = x.len();
    let nstages = logb(n);
    for m in 0..nstages {
        let w = Array::from_iter((0..n / 2).map(|k| { 
                Complex::new(
                   0.0,
                   -2.0 * PI * (brevidx(k, m) as f64) / (2.0f64.powi(m as i32 + 1))
                )
            })) .mapv(|x| x.exp());

        let e=&x.clone().slice_move(s![..(n/2)]);
        let o=&(x.clone().slice_move(s![(n/2)..])*w);
        x.slice_mut(s![0..;2]).assign(&(e+o));
        x.slice_mut(s![1..;2]).assign(&(e-o));
    }

    x.select(Axis(0),&brevidxs(n))
}

In [ ]:
let mut a = Array::from_iter((0..(1<<18)).map(|x| Complex::new(0.0,0.0)));
a[1].re=1.0;
a[4].im=0.25;
let o = fft(&a);
let max = o.fold(0.0,|x,y| y.im.max(x)).max(o.fold(0.0,|x,y| y.re.max(x))) ;
let min = o.fold(0.0,|x,y| y.im.min(x)).min(o.fold(0.0,|x,y| y.re.min(x))) ;
let n=o.len();


let figure = evcxr_figure((640, 480), |root| {
    root.fill(&WHITE)?;
    let mut chart = ChartBuilder::on(&root)
        .caption(format!("FFT output {}",n), ("Arial", 50).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(0.0..(2.0*PI), min..max)?;

    chart.configure_mesh().draw()?;

    chart.draw_series(LineSeries::new(
        (0..n).map(|x| ((x as f64)*2.0*PI/(n as f64),o[x].re)),
        &RED,
    )).unwrap()
        .label("y = o.im")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));
    chart.draw_series(LineSeries::new(
        (0..n).map(|x| ((x as f64) *2.0*PI/(n as f64),o[x].im)),
        &BLUE,
    )).unwrap()
        .label("y = o.re")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));

    chart.configure_series_labels()
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;
    Ok(())
});
figure
